In [1]:
!pip install pyvi

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import pandas as pd
from pyvi import ViTokenizer, ViPosTagger
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *
from sklearn.externals import joblib

Using TensorFlow backend.


In [3]:
df = pd.read_csv("Data_train.csv", encoding = "utf-8")
df.head()

,Unnamed: 0,product,level_1,level_2,target,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1541.0,áo len vàng,Thời trang - Phụ kiện,Thời trang - Phụ kiện không phân biệt giới tính,Áo Nam/Nữ,NaN,NaN,count
1,4041.0,Áo len rớt vai 211 NQ,Thời trang - Phụ kiện,Thời trang nữ,"Áo cardigan, áo len, áo hoodie nữ",NaN,NaN,0
2,4081.0,ao len lo lo pha mau,Thời trang - Phụ kiện,Thời trang - Phụ kiện không phân biệt giới tính,Áo Nam/Nữ,NaN,Thực phẩm - Đồ uống khác,2
3,7421.0,ÁO TD LEN MẮT CƯỜM HỒNG,Thời trang - Phụ kiện,Thời trang nữ,"Áo cardigan, áo len, áo hoodie nữ",NaN,Đồ dùng phòng bếp khác,5
4,7841.0,Áo len sweet cổ viền hoa lụa BG nhí H2043,Thời trang - Phụ kiện,Thời trang nữ,"Áo cardigan, áo len, áo hoodie nữ",NaN,Đồ hộp khác,7


In [4]:
data = df.drop(['Unnamed: 0', 'level_2','level_1', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'] , axis=1)

data.head()

,product,target
0,áo len vàng,Áo Nam/Nữ
1,Áo len rớt vai 211 NQ,"Áo cardigan, áo len, áo hoodie nữ"
2,ao len lo lo pha mau,Áo Nam/Nữ
3,ÁO TD LEN MẮT CƯỜM HỒNG,"Áo cardigan, áo len, áo hoodie nữ"
4,Áo len sweet cổ viền hoa lụa BG nhí H2043,"Áo cardigan, áo len, áo hoodie nữ"


In [5]:
sents = data['product']
sents = sents.str.lower()
X = []
for sent in sents:
    split = ViTokenizer.tokenize( str(sent))
    #print (split)
    X.append(split)

In [6]:
t = data['target']
y = np.array(t)

In [7]:
from sklearn.model_selection import train_test_split
X_data, X_test, y_data, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [8]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(X_data)
X_test_count = count_vect.transform(X_test)

In [9]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)

In [10]:
tfidf_vect.get_feature_names()

['000',
 '0000',
 '0000y000595',
 '001',
 '002',
 '004b',
 '01',
 '017',
 '02',
 '03',
 '031',
 '036',
 '04',
 '05',
 '06',
 '07',
 '0745',
 '08',
 '084',
 '0962',
 '0974',
 '0mg',
 '10',
 '100',
 '1000',
 '10000',
 '10000mah',
 '1000ml',
 '100g',
 '100gr',
 '100k',
 '100ml',
 '100t',
 '1016',
 '102cm',
 '103',
 '105',
 '1060',
 '1091',
 '10g',
 '10inch',
 '10k',
 '10li',
 '10ml',
 '10t',
 '10w',
 '10x',
 '10x17',
 '10x20ml',
 '10x21ml',
 '11',
 '110',
 '110g',
 '110ml',
 '110v',
 '111',
 '1127',
 '115',
 '1150',
 '116231',
 '1166',
 '117',
 '1178',
 '118g',
 '11ly',
 '11ml',
 '12',
 '120',
 '1200',
 '1200whd',
 '120g',
 '120gb',
 '120k',
 '120ml',
 '120v',
 '12196',
 '122',
 '12345',
 '125',
 '125g',
 '125ml',
 '128',
 '1284g',
 '128gb',
 '1299',
 '12d',
 '12g',
 '12m',
 '12t',
 '12x19cm',
 '13',
 '130',
 '130pđ',
 '1339',
 '134',
 '135g',
 '1362',
 '1380ml',
 '139',
 '13ly',
 '14',
 '140128',
 '140g',
 '140ml',
 '1410805',
 '147k',
 '14cm',
 '14k',
 '15',
 '150',
 '15000',
 '150cm',


In [11]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_data)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

In [12]:
tfidf_vect_ngram.get_feature_names()

['000 mah',
 '000 mah cổng_usb',
 '000 mah v_li',
 '000 phút',
 '000 phút gọi',
 '002 túi',
 '002 túi in',
 '004b đồng',
 '01 black',
 '017 gam',
 '02 móc',
 '02 móc treo',
 '02 xám',
 '02 xám caro',
 '03 uha',
 '03 áo',
 '03 áo 240',
 '036 đỏ',
 '036 đỏ 18',
 '04 05',
 '04 05 túi',
 '04 chức',
 '05 03',
 '05 03 áo',
 '05 túi',
 '05 túi da',
 '06 framboiselle',
 '06 hk_037',
 '06 hk_037 thảm',
 '06 philip',
 '07 coral',
 '0745 đá',
 '0745 đá đính',
 '084 29',
 '0962 72',
 '0962 72 1178',
 '0974 16',
 '0974 16 39',
 '10 000',
 '10 000 mah',
 '10 147k',
 '10 bông',
 '10 bông lau',
 '10 chiếc',
 '10 chiếc sữ',
 '10 cm',
 '10 cái',
 '10 dây',
 '10 dẻo',
 '10 dẻo trong',
 '10 hạt',
 '10 hộp',
 '10 hộp cá_mòi',
 '10 hộp sữa_tươi',
 '10 lon',
 '10 lon cá_nục',
 '10 móc',
 '10 móc treo',
 '10 nến',
 '10 nến tealight',
 '10 quần_lót',
 '10 quần_lót nữ',
 '10 quần_lót vera',
 '10 sữa',
 '10 sữa đặc',
 '10 đui_đèn',
 '10 đui_đèn treo',
 '10 đĩa',
 '10 đĩa giấy_uncle',
 '100 chưa',
 '100 chưa acti

In [13]:
# ngram-char level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(X_data)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test)

In [14]:
from sklearn.decomposition import TruncatedSVD

In [15]:
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=42, tol=0.0)

In [16]:
X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

In [17]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=42, tol=0.0)

In [18]:
X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)

In [19]:
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_data_tfidf_ngram_char)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=42, tol=0.0)

In [20]:
X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)

In [21]:
import os
from gensim.models import KeyedVectors 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
word2vec_model_path = os.path.join("Data/vi/vi.vec")

w2v = KeyedVectors.load_word2vec_format(word2vec_model_path)
vocab = w2v.wv.vocab
wv = w2v.wv

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [22]:
def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
#                 print(word)
                sentence.append(wv[word])

        word2vec_data.append(sentence)
#         break
    return word2vec_data

X_data_w2v = get_word2vec_data(X_data)
X_test_w2v = get_word2vec_data(X_test)

In [23]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

In [24]:
encoder.classes_


array(['Balo du lịch, phượt', 'Balo, cặp đựng laptop',
       'Balo, túi, cặp & phụ kiện khác', 'Bàn chải đánh răng', 'Bánh',
       'Bánh kẹo khác', 'Bình đựng nước', 'Bình, ấm, đồ dùng uống trà',
       'Bóng đèn', 'Băng vệ sinh', 'Bật lửa, diêm', 'Bếp gas, bếp lò',
       'Bộ chăm sóc da mặt', 'Ca đựng nước', 'Chiếu', 'Chân váy',
       'Chén đĩa dùng một lần', 'Chăm sóc móng', 'Chăn, mền và phụ kiện',
       'Chảo, chảo chống dính', 'Chống lão hóa', 'Chổi quét nhà, lau nhà',
       'Cài Áo', 'Cá hộp', 'Dung dịch vệ sinh nhà tắm',
       'Dây chuyền, vòng cổ, mặt dây chuyền nữ', 'Dưỡng Thể',
       'Dưỡng trắng', 'Dưỡng tóc, ủ tóc', 'Dưỡng ẩm', 'Dầu gội, dầu xả',
       'Dầu ăn', 'Dụng cụ cạo râu', 'Dụng cụ diệt côn trùng khác',
       'Dụng cụ sửa chữa khác', 'Dụng cụ trang điểm', 'Dụng cụ tẩy trang',
       'Gia vị dạng bột', 'Giày boots nữ', 'Giày búp bê', 'Giày cao gót',
       'Giày dép Nam/Nữ', 'Giày dép bé gái', 'Giày dép nam khác',
       'Giày dép nữ khác', 'Giày sneakers n

In [25]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))

In [26]:
train_model(naive_bayes.MultinomialNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

Validation accuracy:  0.06018518518518518
Test accuracy:  0.06319702602230483


In [27]:
train_model(naive_bayes.BernoulliNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

Validation accuracy:  0.023148148148148147
Test accuracy:  0.02973977695167286


In [28]:
train_model(linear_model.LogisticRegression(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Validation accuracy:  0.16203703703703703
Test accuracy:  0.24349442379182157


In [29]:
train_model(svm.SVC(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Validation accuracy:  0.023148148148148147
Test accuracy:  0.02973977695167286


In [30]:
train_model(ensemble.RandomForestClassifier(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Validation accuracy:  0.4722222222222222
Test accuracy:  0.4962825278810409


In [31]:

def create_dnn_model():
    input_layer = Input(shape=(300,))
    layer = Dense(1024, activation='relu')(input_layer)
    layer = Dense(1024, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [32]:

def create_brnn_model():
    input_layer = Input(shape=(300,))
    
    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu'))(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [34]:
#create_brnn_model(X_data_tfidf, y_data_n, X_test_tfidf, n_epochs=3)

In [35]:
from sklearn import tree

In [36]:
clf = tree.DecisionTreeClassifier()

In [37]:
train_model(clf, X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

Validation accuracy:  0.49074074074074076
Test accuracy:  0.4758364312267658
